In [1]:
import psycopg2
import pandas as pd
import io
import os

In [2]:
def database(db_command_func, *data):
    try:
        #! 换成自己的用户名密码
        conn = psycopg2.connect(dbname='mikedb', user='mike', host='104.198.59.118', password='')
        print("connection success")
    except:
        print("I am unable to connect to the database")
        
    result = 0
    if not data:    
        result = db_command_func(conn.cursor())
    else:
        result = db_command_func(conn.cursor(), *data)
        
    conn.commit()
    conn.close()
    
    return result

In [3]:
def create_table_if_not_exist(cursor):
    cursor.execute("""
                CREATE TABLE IF NOT EXISTS sz_stocks (
                        pricing_date TIMESTAMP NOT NULL, 
                        open  DECIMAL NOT NULL, 
                        high  DECIMAL NOT NULL, 
                        low  DECIMAL NOT NULL,
                        close  DECIMAL NOT NULL,
                        volume  DECIMAL NOT NULL,
                        stock_symbol VARCHAR(255) NOT NULL)""")
    #CURRENT_TIMESTAMP
    print ("Table create successful!")

In [4]:
database(create_table_if_not_exist)

connection success
Table create successful!


In [5]:
def insert_value(cursor, df):
    output = io.StringIO();
# ignore the index
    df.to_csv(output, sep='\t',index = False, header = False);
    output.getvalue();
# jump to start of stream
    output.seek(0);
    cursor.copy_from(output, 'sz_stocks', null='')
    print ("Data insert successful!")
# data frame to sql, df.to_sql('db_table2', engine, if_exists='replace')

In [6]:
def select_table_name(cursor):
    cursor.execute("""SELECT table_schema, table_name FROM information_schema.tables
                       WHERE table_schema = 'public'""")
    rows = cursor.fetchall()
    print ("\nShow me the data:\n")

    # inserted line sum 5000 * 500, head(0) 
    for row in rows:
        print ("   ", row)

In [7]:
database(select_table_name)

connection success

Show me the data:

    ('public', 'company')
    ('public', 'sz_stocks')
    ('public', 'company_example')
    ('public', 'stock_example')
    ('public', 'exchange_example')


In [12]:
def read_csv(folder_path):
    try:
        file_list = os.listdir(folder_path)
    except:
        print('cannot find the folder path')
    # file_list line sum -> 5000 * 500, head(0) random sample 
    count = 0
    print('total '+ str(len(file_list)) + ' files')
    for name in file_list:
        file_path = folder_path + '/' + name
        df0 = pd.read_csv(file_path, sep = ',' ,names = ['date','open','high','low','close','volume','name'],index_col=False,header=None)
        df = df0.drop(df0.index[0])
        df['name'] = name[0:8]
        database(insert_value, df)
        count += 1
        speed = round(count / len(file_list), 3)
        print('file No. ' + str(count) + '; progress bar: '+ str(speed))
    print ("All data insert successful")

read_csv(os.getcwd() + '/stocks')    

total 4 files
connection success
Data insert successful!
file No. 1; progress bar: 0.25
connection success
Data insert successful!
file No. 2; progress bar: 0.5
connection success
Data insert successful!
file No. 3; progress bar: 0.75
connection success
Data insert successful!
file No. 4; progress bar: 1.0
All data insert successful


In [15]:
def select_data(cursor):
    cursor.execute("""SELECT pricing_date, close, stock_symbol from sz_stocks WHERE pricing_date < '2018-01-01'  and pricing_date > '2007-12-31'""")
    rows = cursor.fetchall()
    cursor.execute("""SELECT COUNT(*) from sz_stocks""")
    count = cursor.fetchall()
    print ("\nShow me the data:\n")
    print('Total' + str(count) + ' data are selected')
    return rows

        
def select_table_name(cursor):
    cursor.execute("""SELECT table_schema, table_name FROM information_schema.tables
                       WHERE table_schema = 'public'""")
    rows = cursor.fetchall()
    print ("\nShow me the data:\n")

    # inserted line sum 5000 * 500, head(0) 
    for row in rows:
        print ("   ", row)


In [18]:
folder_path = os.getcwd() + '/stocks'
database(create_table_if_not_exist)
read_csv(folder_path)
database(select_table_name)
a = database(select_data)
print(a)
database(check_insert, folder_path)


connection success
Table create successful!
total 4 files
connection success
Data insert successful!
file No. 1; progress bar: 0.25
connection success
Data insert successful!
file No. 2; progress bar: 0.5
connection success
Data insert successful!
file No. 3; progress bar: 0.75
connection success
Data insert successful!
file No. 4; progress bar: 1.0
All data insert successful
connection success

Show me the data:

    ('public', 'company')
    ('public', 'sz_stocks')
    ('public', 'company_example')
    ('public', 'stock_example')
    ('public', 'exchange_example')
connection success

Show me the data:

Total[(85,)] data are selected
[(datetime.datetime(2012, 12, 31, 0, 0), Decimal('5.500'), 'SZ430129'), (datetime.datetime(2012, 9, 7, 0, 0), Decimal('5.900'), 'SZ430139'), (datetime.datetime(2012, 10, 19, 0, 0), Decimal('6.280'), 'SZ430139'), (datetime.datetime(2013, 1, 18, 0, 0), Decimal('5.000'), 'SZ430139'), (datetime.datetime(2013, 2, 26, 0, 0), Decimal('6.000'), 'SZ430139'), (date

In [21]:
def check_insert(cursor, folder_path):
    cursor.execute("""SELECT * from sz_stocks WHERE TO_CHAR(pricing_date,'YYYY/MM/DD') = '2013/06/17'""")
    rows = cursor.fetchall()
    test_id = 'SZ430149'
    test_1 = 0
    test_2 = 0
    for row in rows:
        if row[-2] == test_id:
            test_1 += 1
            sql_close = row[-4]
    try:
        file_list = os.listdir(folder_path)
    except:
        print('cannot find the folder path')
    # file_list line sum -> 5000 * 500, head(0) random sample 
    for name in file_list:
        if name == test_id:
            file_path = folder_path + '/' + name
            df0 = pd.read_csv(file_path, sep = ',' ,names = ['date','open','high','low','close','volume','name'],index_col=False,header=None)
            df = df0.drop(df0.index[0])
            
            for _, row in df.iterrows():
                if row.date == '2017/12/1 0:00':
                    test_2 += 1
                    csv_close = row.close
    print ('sql ' + str(test_1) + '; csv ' + str(test_2) + str(test_2))
    print ('sql_close: ' + str(sql_close) + '; csv_close: ' + str(csv_close))
    if sql_close == csv_close:
        print ('Test PASS ')  

In [22]:
database(check_insert, folder_path)

connection success
sql 0; csv 00


UnboundLocalError: local variable 'sql_close' referenced before assignment